# Preparation 
[tutorial for pp-structure layout training](https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.6/ppstructure/layout/README.md#51-Train)

```
category_map = {
    "heading": 0,
    "question": 1, 
    "subquestion": 2,
    "choice"  : 3,
    "image"   : 4,
    "table"   : 5,
    "blank"   : 6,
    "auxillary_text"   : 7,
    
}
```

In [1]:
%cd /content

/content


In [2]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown -O coco.zip --id 1qNpaKvCyotOj43Z4FrQhamMetzSiDPQQ
!unzip coco.zip

# **Env**

## conda

In [ ]:
!apt install python3.8
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!update-alternatives --set python3 /usr/bin/python3.8
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python get-pip.py
!pip --version
!python3 -m pip install setuptools==59.5.0
# !pip install virtualenv
# !python3 -m virtualenv venv


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.8 is already the newest version (3.8.10-0ubuntu1~20.04.7).
python3.8 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in manual mode
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2518k  100 2518k    0     0  20.6M      0 --:--:-- --:--:-- --:--:-- 20.6M
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 7.2 MB/s eta 0:00:00
pip 23.1.2 from /usr/local/lib/python3.8/dist-packages/pip (pyt

## pip

In [ ]:
!pip  install --upgrade pip

# GPU Install
# !source /conntent/venv/bin/activate; pip install "paddlepaddle-gpu>=2.3" -i https://mirror.baidu.com/pypi/simple

# CPU Install
!python -m pip install "paddlepaddle>=2.3" -i https://mirror.baidu.com/pypi/simple

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git --branch develop

In [ ]:
%cd PaddleDetection
!pip install -r requirements.txt

> modify
```configs/picodet/legacy_model/application/layout_analysis/picodet_lcnet_x1_0_layout.yml``` as an example, the change is as follows:

```
metric: COCO
# Number of categories
num_classes: 8

TrainDataset:
  !COCODataSet
    # Modify to your own training data directory
    image_dir: train
    # Modify to your own training data label file
    anno_path: train.json
    # Modify to your own training data root directory
    dataset_dir: /content/dataset/questiondetection
    data_fields: ['image', 'gt_bbox', 'gt_class', 'is_crowd']

EvalDataset:
  !COCODataSet
    # Modify to your own validation data directory
    image_dir: val
    # Modify to your own validation data label file
    anno_path: val.json
    # Modify to your own validation data root
    dataset_dir: /content/dataset/questiondetection

TestDataset:
  !ImageFolder
    # Modify to your own test data label file
    anno_path: /content/dataset/questiondetection/test.json

```

In [ ]:
!source /content/venv/bin/activate; pip install numba==0.56.4

# **Train**

In [ ]:
%cd /content

/content


In [ ]:
!mkdir pretrained_model
%cd pretrained_model
# Download PubLayNet pre-training model（Direct experience model evaluates, predicts, and turns static）
!wget https://bj.bcebos.com/v1/paddledet/models/rtdetr_hgnetv2_x_6x_coco.pdparams
# Download the PubLaynet inference model（Direct experience model reasoning）
# !wget https://paddleocr.bj.bcebos.com/ppstructure/models/layout/picodet_lcnet_x1_0_fgd_layout_infer.tar

In [ ]:
#@title basic configure
cfg = """
epoch: 72

LearningRate:
  base_lr: 0.0001
  schedulers:
  - !PiecewiseDecay
    gamma: 1.0
    milestones: [100]
    use_warmup: true
  - !LinearWarmup
    start_factor: 0.001
    steps: 2000

OptimizerBuilder:
  clip_grad_by_norm: 0.1
  regularizer: false
  optimizer:
    type: AdamW
    weight_decay: 0.0001

architecture: DETR
pretrain_weights: https://paddledet.bj.bcebos.com/models/pretrained/ResNet50_vd_ssld_v2_pretrained.pdparams
norm_type: sync_bn
use_ema: True
ema_decay: 0.9999
ema_decay_type: "exponential"
ema_filter_no_grad: True
hidden_dim: 256
use_focal_loss: True
eval_size: [640, 640]


DETR:
  backbone: ResNet
  neck: HybridEncoder
  transformer: RTDETRTransformer
  detr_head: DINOHead
  post_process: DETRPostProcess

ResNet:
  # index 0 stands for res2
  depth: 50
  variant: d
  norm_type: bn
  freeze_at: 0
  return_idx: [1, 2, 3]
  lr_mult_list: [0.1, 0.1, 0.1, 0.1]
  num_stages: 4
  freeze_stem_only: True

HybridEncoder:
  hidden_dim: 256
  use_encoder_idx: [2]
  num_encoder_layers: 1
  encoder_layer:
    name: TransformerLayer
    d_model: 256
    nhead: 8
    dim_feedforward: 1024
    dropout: 0.
    activation: 'gelu'
  expansion: 1.0


RTDETRTransformer:
  num_queries: 300
  position_embed_type: sine
  feat_strides: [8, 16, 32]
  num_levels: 3
  nhead: 8
  num_decoder_layers: 6
  dim_feedforward: 1024
  dropout: 0.0
  activation: relu
  num_denoising: 100
  label_noise_ratio: 0.5
  box_noise_scale: 1.0
  learnt_init_query: False

DINOHead:
  loss:
    name: DINOLoss
    loss_coeff: {class: 1, bbox: 5, giou: 2}
    aux_loss: True
    use_vfl: True
    matcher:
      name: HungarianMatcher
      matcher_coeff: {class: 2, bbox: 5, giou: 2}

DETRPostProcess:
  num_top_queries: 300

worker_num: 4
TrainReader:
  sample_transforms:
    - Decode: {}
    - RandomDistort: {}
    - RandomExpand: {} #{fill_value: [123.675, 116.28, 103.53]}
    - RandomCrop: {}
    - RandomFlip: {prob: 0.8}
  batch_transforms:
    - BatchRandomResize: {target_size: [480, 512, 544, 576, 608, 640, 640, 640, 672, 704, 736, 768, 800], random_size: True, random_interp: True, keep_ratio: False}
    - NormalizeImage: {mean: [0., 0., 0.], std: [1., 1., 1.], norm_type: none}
    - NormalizeBox: {}
    - BboxXYXY2XYWH: {}
    - Permute: {}
  batch_size: 4
  shuffle: true
  drop_last: true
  collate_batch: false
  use_shared_memory: false


EvalReader:
  sample_transforms:
    - Decode: {}
    - Resize: {target_size: [640, 640], keep_ratio: False, interp: 2}
    - NormalizeImage: {mean: [0., 0., 0.], std: [1., 1., 1.], norm_type: none}
    - Permute: {}
  batch_size: 4
  shuffle: false
  drop_last: false


TestReader:
  inputs_def:
    image_shape: [3, 640, 640]
  sample_transforms:
    - Decode: {}
    - Resize: {target_size: [640, 640], keep_ratio: False, interp: 2}
    - NormalizeImage: {mean: [0., 0., 0.], std: [1., 1., 1.], norm_type: none}
    - Permute: {}
  batch_size: 1
  shuffle: false
  drop_last: false
"""

with open("/content/base.yaml", "w") as f: f.write(cfg)

In [ ]:
cfg = """
_BASE_: [
  '/content/PaddleDetection/configs/datasets/coco_detection.yml',
  '/content/base.yaml',
]

weights: /content/pretrained_model/rtdetr_hgnetv2_x_6x_coco
find_unused_parameters: True
log_iter: 200

HybridEncoder:
  hidden_dim: 256
  use_encoder_idx: [2]
  num_encoder_layers: 1
  encoder_layer:
    name: TransformerLayer
    d_model: 256
    nhead: 8
    dim_feedforward: 1024
    dropout: 0.
    activation: 'gelu'
  expansion: 0.5
  depth_mult: 1.0

RTDETRTransformer:
  eval_idx: 2 # use 3th decoder layer to eval

use_gpu: flase
use_xpu: false
use_mlu: false
use_npu: false
log_iter: 20
save_dir: output
snapshot_epoch: 1
print_flops: false
print_params: false

# Exporting the model
export:
  post_process: True  # Whether post-processing is included in the network when export model.
  nms: True           # Whether NMS is included in the network when export model.
  benchmark: False    # It is used to testing model performance, if set `True`, post-process and NMS will not be exported.
  fuse_conv_bn: False
"""
with open("/content/rtDertX.yaml", "w") as f: f.write(cfg)

In [ ]:
# Single card training
%cd /content/PaddleDetection
!export CUDA_VISIBLE_DEVICES=-1
!python tools/train.py \
    -c /content/rtDertX.yaml \
    --eval

Exception in thread Thread-2:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/paddle/fluid/dataloader/dataloader_iter.py", line 623, in _get_data
    data = self._data_queue.get(timeout=self._timeout)
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 108, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/paddle/fluid/dataloader/dataloader_iter.py", line 536, in _thread_loop
    batch = self._get_data()
  File "/usr/local/lib/python3.8/dist-packages/paddle/fluid/dataloader/dataloader_iter.py", line 638, in _get_data
    raise RuntimeError("DataLoader {} workers exit unexpectedly, " \
RuntimeError: DataLoader 1 workers e

# **TEST**

In [ ]:
%cd /content

/content


In [ ]:
!pip install gdown
!gdown -O data.zip  --id 1qNpaKvCyotOj43Z4FrQhamMetzSiDPQQ
!gdown -O best_model.pdparams --id 1mWUyf8jnyUuQjJlNINio3tJCdSnzpSXD
!gdown -O testds.zip --id 1aRry9bdEPSO83JQwvZ_By5Xyp2FQBVyx
!gdown -O searchable.zip --id 1h0YlStQZzdVhYoXjXBVgNf7oMQU6tBe7

In [ ]:
!unzip data.zip
!unzip testds.zip
!unzip searchable.zip

In [ ]:
%cd PaddleDetection/

/content/PaddleDetection


In [ ]:
!pip install pycocotools

In [ ]:
!python tools/export_model.py \
    -c /content/cfg.yaml \
    -o weights=/content/best_model \
    --output_dir=/content/output_inference/

In [ ]:
!python tools/infer.py \
    -c /content/cfg.yaml \
    -o weights='/content/best_model.pdparams' \
    --output_dir=/content/output_dir/ \
    --draw_threshold=0.5 \
    --infer_dir="/content/small_test" \
    #--infer_img='/content/content/CoCodata/images/test/searchable843_0.png' \

# **ZIP**

In [ ]:
!zip out.zip /content/output_dir -r